In [3]:
# Install necessary packages
# pandas , azure-storage-blob, dotenv
!pip install pandas azure-storage-blob python-dotenv

  Using cached azure_storage_blob-12.25.1-py3-none-any.whl.metadata (26 kB)
  Using cached azure_core-1.34.0-py3-none-any.whl.metadata (42 kB)
  Using cached isodate-0.7.2-py3-none-any.whl.metadata (11 kB)
Using cached azure_storage_blob-12.25.1-py3-none-any.whl (406 kB)
Using cached azure_core-1.34.0-py3-none-any.whl (207 kB)
Using cached isodate-0.7.2-py3-none-any.whl (22 kB)


In [18]:
# Importing necessary libraries
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os

In [19]:
# Data Extraction

try:
    data = pd.read_json(r'MarketsData.json')
    print('Data extracted successfully')
except Exception as e:
    print(f'Error extracting data: {e}')

Data extracted successfully


In [20]:
data.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,107598.00,2137849603370,1,2137849603370,28668694414,109071.00,...,1.986980e+07,2.100000e+07,111814.00,-3.75094,2025-05-22T18:41:28.492Z,67.810000,1.586106e+05,2013-07-06T00:00:00.000Z,None,2025-05-25T20:37:23.382Z
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,2521.01,304290661820,2,304290661820,13450389192,2554.59,...,1.207260e+08,NaN,4878.26,-48.34656,2021-11-10T14:24:19.604Z,0.432979,5.818659e+05,2015-10-20T00:00:00.000Z,"{'times': 30.32373818665742, 'currency': 'btc'...",2025-05-25T20:37:23.056Z
2,tether,usdt,Tether,https://coin-images.coingecko.com/coins/images...,1.00,152777148448,3,152777148448,45624245689,1.00,...,1.527330e+11,NaN,1.32,-24.39731,2018-07-24T00:00:00.000Z,0.572521,7.471772e+01,2015-03-02T00:00:00.000Z,None,2025-05-25T20:37:23.139Z
3,ripple,xrp,XRP,https://coin-images.coingecko.com/coins/images...,2.30,135192724252,4,230334194294,1725587726,2.35,...,9.998611e+10,1.000000e+11,3.40,-32.19464,2018-01-07T00:00:00.000Z,0.002686,8.568384e+04,2014-05-22T00:00:00.000Z,None,2025-05-25T20:37:22.786Z
4,binancecoin,bnb,BNB,https://coin-images.coingecko.com/coins/images...,667.39,97364572479,5,97364572479,760237143,674.99,...,1.458876e+08,2.000000e+08,788.84,-15.38177,2024-12-04T10:35:25.220Z,0.039818,1.676304e+06,2017-10-19T00:00:00.000Z,None,2025-05-25T20:37:21.303Z


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                100 non-null    object 
 1   symbol                            100 non-null    object 
 2   name                              100 non-null    object 
 3   image                             100 non-null    object 
 4   current_price                     100 non-null    float64
 5   market_cap                        100 non-null    int64  
 6   market_cap_rank                   100 non-null    int64  
 7   fully_diluted_valuation           100 non-null    int64  
 8   total_volume                      100 non-null    int64  
 9   high_24h                          100 non-null    float64
 10  low_24h                           100 non-null    float64
 11  price_change_24h                  100 non-null    float64
 12  price_cha

In [61]:
# Data cleaning and transformation
# Handling missing values ( filling missing numeric values with mean or median )
numeric_columns = data.select_dtypes(include=['float64', 'Int64']).columns
for col in numeric_columns:
    data.fillna({col: data[col].mean()}, inplace=True)


In [62]:
# Handling missing values ( filling missing object/string values with 'Unknown' )
string_columns = data.select_dtypes(include=['object']).columns
for col in string_columns:
    data.fillna({col: 'Unknown'}, inplace=True)


In [63]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                100 non-null    object 
 1   symbol                            100 non-null    object 
 2   name                              100 non-null    object 
 3   image                             100 non-null    object 
 4   current_price                     100 non-null    float64
 5   market_cap                        100 non-null    int64  
 6   market_cap_rank                   100 non-null    int64  
 7   fully_diluted_valuation           100 non-null    int64  
 8   total_volume                      100 non-null    int64  
 9   high_24h                          100 non-null    float64
 10  low_24h                           100 non-null    float64
 11  price_change_24h                  100 non-null    float64
 12  price_cha

In [64]:
data.columns

Index(['id', 'symbol', 'name', 'image', 'current_price', 'market_cap',
       'market_cap_rank', 'fully_diluted_valuation', 'total_volume',
       'high_24h', 'low_24h', 'price_change_24h',
       'price_change_percentage_24h', 'market_cap_change_24h',
       'market_cap_change_percentage_24h', 'circulating_supply',
       'total_supply', 'max_supply', 'ath', 'ath_change_percentage',
       'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi',
       'last_updated'],
      dtype='object')

In [66]:
# creating a crypto_assets table
crypto_assets = data[['name', 'symbol', 'current_price', 'market_cap', 'total_volume', 'price_change_percentage_24h']].drop_duplicates().reset_index(drop=True)
crypto_assets.index.name = 'asset_id'
crypto_assets = crypto_assets.reset_index()

In [67]:
crypto_assets.head()

,asset_id,name,symbol,current_price,market_cap,total_volume,price_change_percentage_24h
0,0,Bitcoin,btc,107598.00,2137849603370,28668694414,-1.35079
1,1,Ethereum,eth,2521.01,304290661820,13450389192,-1.30773
2,2,Tether,usdt,1.00,152777148448,45624245689,0.00841
3,3,XRP,xrp,2.30,135192724252,1725587726,-1.94777
4,4,BNB,bnb,667.39,97364572479,760237143,-1.12717


In [68]:
# create crypto_market table
crypto_market = data[['market_cap_rank', 'high_24h', 'low_24h', 'price_change_24h', 'price_change_percentage_24h']].drop_duplicates().reset_index(drop=True)
crypto_market.index.name = 'market_id'
crypto_market = crypto_market.reset_index()

In [69]:
crypto_market.head()

,market_id,market_cap_rank,high_24h,low_24h,price_change_24h,price_change_percentage_24h
0,0,1,109071.00,106802.00,-1473.319665,-1.35079
1,1,2,2554.59,2479.53,-33.404854,-1.30773
2,2,3,1.00,1.00,0.000084,0.00841
3,3,4,2.35,2.28,-0.045759,-1.94777
4,4,5,674.99,657.95,-7.608362,-1.12717


In [70]:
# create crypto_prices table
crypto_prices = data[['current_price', 'price_change_24h', 'price_change_percentage_24h']].drop_duplicates().reset_index(drop=True)
crypto_prices.index.name = 'price_id'
crypto_prices = crypto_prices.reset_index()

In [54]:
crypto_prices.head()

,price_id,current_price,price_change_24h,price_change_percentage_24h
0,0,107598.00,-1473.319665,-1.35079
1,1,2521.01,-33.404854,-1.30773
2,2,1.00,0.000084,0.00841
3,3,2.30,-0.045759,-1.94777
4,4,667.39,-7.608362,-1.12717


In [71]:
# create crypto datetime table
crypto_datetime = data[['name', 'last_updated']].drop_duplicates().reset_index(drop=True)
crypto_datetime.index.name = 'datetime_id'
crypto_datetime = crypto_datetime.reset_index()

In [60]:
crypto_datetime.head()

,datetime_id,name,last_updated
0,0,Bitcoin,2025-05-25T20:37:23.382Z
1,1,Ethereum,2025-05-25T20:37:23.056Z
2,2,Tether,2025-05-25T20:37:23.139Z
3,3,XRP,2025-05-25T20:37:22.786Z
4,4,BNB,2025-05-25T20:37:21.303Z


In [72]:
# transaction table
transaction = data.merge(crypto_assets, on=['name', 'symbol', 'current_price', 'market_cap', 'total_volume', 'price_change_percentage_24h'], how='left') \
                   .merge(crypto_market, on=['market_cap_rank', 'high_24h', 'low_24h', 'price_change_24h', 'price_change_percentage_24h'], how='left') \
                   .merge(crypto_prices, on=['current_price', 'price_change_24h', 'price_change_percentage_24h'], how='left') \
                   .merge(crypto_datetime, on=['name', 'last_updated'], how='left')

transaction.index.name = 'transaction_id'
transaction = transaction.reset_index() \
                         [[ 'transaction_id', 'asset_id', 'market_id', 'price_id', 'datetime_id', 
                           'name', 'symbol', 'current_price', 'market_cap', 'total_volume', 
                           'price_change_percentage_24h', 'market_cap_rank', 'high_24h', 
                           'low_24h', 'price_change_24h', 'last_updated']]

In [73]:
transaction.head()

,transaction_id,asset_id,market_id,price_id,datetime_id,name,symbol,current_price,market_cap,total_volume,price_change_percentage_24h,market_cap_rank,high_24h,low_24h,price_change_24h,last_updated
0,0,0,0,0,0,Bitcoin,btc,107598.00,2137849603370,28668694414,-1.35079,1,109071.00,106802.00,-1473.319665,2025-05-25T20:37:23.382Z
1,1,1,1,1,1,Ethereum,eth,2521.01,304290661820,13450389192,-1.30773,2,2554.59,2479.53,-33.404854,2025-05-25T20:37:23.056Z
2,2,2,2,2,2,Tether,usdt,1.00,152777148448,45624245689,0.00841,3,1.00,1.00,0.000084,2025-05-25T20:37:23.139Z
3,3,3,3,3,3,XRP,xrp,2.30,135192724252,1725587726,-1.94777,4,2.35,2.28,-0.045759,2025-05-25T20:37:22.786Z
4,4,4,4,4,4,BNB,bnb,667.39,97364572479,760237143,-1.12717,5,674.99,657.95,-7.608362,2025-05-25T20:37:21.303Z


In [77]:
transaction['date'] = pd.to_datetime(transaction['last_updated'])

In [78]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   transaction_id               100 non-null    int64              
 1   asset_id                     100 non-null    int64              
 2   market_id                    100 non-null    int64              
 3   price_id                     100 non-null    int64              
 4   datetime_id                  100 non-null    int64              
 5   name                         100 non-null    object             
 6   symbol                       100 non-null    object             
 7   current_price                100 non-null    float64            
 8   market_cap                   100 non-null    int64              
 9   total_volume                 100 non-null    int64              
 10  price_change_percentage_24h  100 non-null    float6

In [79]:
# Saving the data to CSV files
data.to_csv('cleaned_data.csv', index=False)
crypto_assets.to_csv('crypto_assets.csv', index=False)
crypto_market.to_csv('crypto_market.csv', index=False)
crypto_prices.to_csv('crypto_prices.csv', index=False)
crypto_datetime.to_csv('crypto_datetime.csv', index=False)
transaction.to_csv('transaction.csv', index=False)